## TITLE

In [4]:
from LLMs_attack import *
import pickle

print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Doit afficher True
print(torch.version.cuda)  # Doit afficher une version de CUDA
print(torch.cuda.get_device_name(0))  # Doit afficher le nom de votre GPU

2.6.0+cu118
True
11.8
NVIDIA GeForce RTX 4060 Laptop GPU


In [5]:
args = argparse.Namespace(
ntrain=0,
data_dir="data/MMLU",
engine=["qwen1"],
n_reduced=None,
use_subset=True,
subset_size = 2,
permutation_attack=False,
position_permute=False,
reduce_attack=False,
load_in_8bit=False,
verbose = True,
stop_attack = False,
subset_subjects = 2,
)

In [ ]:
# main(args)

['abstract_algebra', 'elementary_mathematics']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1'], n_reduced=None, use_subset=True, subset_size=2, permutation_attack=False, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=True, stop_attack=False, subset_subjects=2)
Engine: qwen1


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.



Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A. 0
B. 4
C. 2
D. 6
Answer:

Predicted:  B
True:  B
Probabilities:  [1.1324882507324219e-06, 3.4749507904052734e-05, 2.682209014892578e-06, 1.9669532775878906e-06]


Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.
A. 8
B. 2
C. 24
D. 120
Answer:

Predicted:  B
True:  C
Probabilities:  [9.894371032714844e-06, 2.0444393157958984e-05, 1.9669532775878906e-06, 8.64267349243164e-06]

Average accuracy 50.00 - abstract_algebra

Prompt:  The following are multiple choice questions (with answers) about elementary_mathematics.

What is the value of p in 24 = 2p?
A. p = 4
B. p = 8
C. p = 12
D. p = 24
Answer:

Predicted:  B
True:  C
Probabilities:  [1.055002212524414e-05, 2.104043960571289e-05, 5.185604095458984e-06

({'qwen1': [np.float64(0.5), np.float64(0.5)]}, {})

In [6]:
args.permutation_attack = True
args.subset_size = 1
args.subset_subjects = 1

In [ ]:
# main(args)

['abstract_algebra']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1'], n_reduced=None, use_subset=True, subset_size=1, permutation_attack=True, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=True, stop_attack=False, subset_subjects=1)
Engine: qwen1
#####################################################################
Question:  Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
Permutation:  0

Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A. 0
B. 4
C. 2
D. 6
Answer:

Predicted:  B
True:  B
Probabilities:  [1.1324882507324219e-06, 3.4749507904052734e-05, 2.682209014892578e-06, 1.9669532775878906e-06]

Permutation:  1

Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over 

({'qwen1': []}, {'qwen1': [array([0.375])]})

In [11]:
args.subset_size = 101
args.subset_subjects = 15
args.verbose = False
args.permutation_attack = False
args.stop_attack = True
args.engine = ["qwen1", "llama-3b"]
accuracy = {}

In [ ]:
accuracy['standard'] = main(args)

['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1', 'llama-3b'], n_reduced=None, use_subset=True, subset_size=101, permutation_attack=False, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=False, stop_attack=True, subset_subjects=15)
Engine: qwen1
Average accuracy 36.00 - abstract_algebra
Average accuracy 23.76 - anatomy
Average accuracy 38.61 - astronomy
Average accuracy 41.00 - business_ethics
Average accuracy 39.60 - clinical_knowledge
Average accuracy 28.71 - college_biology
Average accuracy 42.00 - college_chemistry
Average accuracy 38.00 - college_computer_science
Average accuracy 42.00 - college_mathematics
Average accuracy 33.66 - college_medicine
Ave

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Average accuracy 35.00 - abstract_algebra
Average accuracy 62.38 - anatomy
Average accuracy 66.34 - astronomy
Average accuracy 55.00 - business_ethics
Average accuracy 67.33 - clinical_knowledge
Average accuracy 74.26 - college_biology
Average accuracy 42.00 - college_chemistry
Average accuracy 47.00 - college_computer_science
Average accuracy 34.00 - college_mathematics
Average accuracy 61.39 - college_medicine
Average accuracy 37.62 - college_physics
Average accuracy 65.00 - computer_security
Average accuracy 51.49 - conceptual_physics
Average accuracy 41.58 - econometrics
Average accuracy 51.49 - electrical_engineering
Average accuracy: 52.82





In [15]:
# WARNING : WILL OVERRRIDE THE PREVIOUS FILE

with open("accuracy.pkl", "wb") as f:
    pickle.dump(accuracy, f)
    print("accuracy.pkl saved")

accuracy.pkl saved


In [13]:
if accuracy == {}:
    try:
        with open("accuracy.pkl", "rb") as f:
            accuracy = pickle.load(f)
            print("Loaded accuracy.pkl")
    except:
        print("Could not load accuracy.pkl")

print(accuracy)
print(accuracy['standard'][0]['qwen1'][0])

Loaded accuracy.pkl
{'standard': ({'qwen1': [np.float64(0.36), np.float64(0.2376237623762376), np.float64(0.38613861386138615), np.float64(0.41), np.float64(0.39603960396039606), np.float64(0.2871287128712871), np.float64(0.42), np.float64(0.38), np.float64(0.42), np.float64(0.33663366336633666), np.float64(0.3069306930693069), np.float64(0.36), np.float64(0.3564356435643564), np.float64(0.25742574257425743), np.float64(0.37623762376237624)], 'llama-3b': [np.float64(0.35), np.float64(0.6237623762376238), np.float64(0.6633663366336634), np.float64(0.55), np.float64(0.6732673267326733), np.float64(0.7425742574257426), np.float64(0.42), np.float64(0.47), np.float64(0.34), np.float64(0.6138613861386139), np.float64(0.37623762376237624), np.float64(0.65), np.float64(0.5148514851485149), np.float64(0.4158415841584158), np.float64(0.5148514851485149)]}, {})}
0.36


In [ ]:
args.permutation_attack = True
accuracy['attack'] = main(args)


In [ ]:
args.subset_size = 50
args.subset_subjects = 3
args.permutation_attack = True
args.stop_attack = False # Compute the results for all permutation
accuracy['full_attack'] = main(args)